In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('image', cmap='gray')
import matplotlib.animation

%matplotlib ipympl
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
import cv2
from tqdm import tqdm
import random
from stardist import _draw_polygons
from stardist.models import StarDist2D
from csbdeep.utils import normalize
import skimage
from scipy.optimize import dual_annealing, minimize
from tifffile import imsave, imread
from utils import filter_detection_data, get_frame #detect_features,
import trackpy as tp

In [2]:
#model_name = 'stardist_trained'          # stardist model trained for 50 epochs on simulated synthetic dataset
model_name = 'modified_2D_versatile_fluo' # stardist model trained for 150 epochs on simulated dataset starting from the pretrained 2D versatile fluo model
model = StarDist2D(None, name = model_name, basedir = './models/')

video_selection = "25b25r-1" # "49b1r"
if video_selection == "25b25r-1":
    xmin, ymin, xmax, ymax = 95, 30, 535, 470    
elif video_selection == "49b1r":
    xmin, ymin, xmax, ymax = 20, 50, 900, 930

save_path       = f'./{video_selection}/{model_name}/'
source_path     = f'./data/{video_selection}.mp4'
system_name     = f'{video_selection} system'
nDrops = 50

video = cv2.VideoCapture(source_path)
video.set(cv2.CAP_PROP_POS_FRAMES, 0)
w = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video.get(cv2.CAP_PROP_FPS))
n_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Video has {n_frames} frames with a resolution of {w}x{h} and a framerate of {fps} fps")

Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.989263, nms_thresh=0.3.
Video has 540000 frames with a resolution of 640x480 and a framerate of 30 fps


In [7]:
test_verb = False
detect_verb = False
link_verb = False
interp_verb = False

startFrame = 0
endFrame = 539999

In [8]:
if test_verb: 
    n_samples = 100
    test_detection(n_samples, n_frames, nDrops, video_selection, model, model_name, video, xmin, ymin, xmax, ymax, w, h, save_path)

In [9]:
if detect_verb:
    print(f'Processing from {int(startFrame/fps)} s to {int(endFrame/fps)} s')
    sample_frames = np.arange(startFrame, endFrame, 1, dtype=int)
    raw_detection_df = detect_features(sample_frames, False, video_selection, model, model_name, video, xmin, ymin, xmax, ymax, w, h, save_path)
    
    n_feature_per_frame = raw_detection_df.groupby('frame').count().x.values
    fig, ax = plt.subplots(2, 2, figsize = (8, 4))
    ax[0, 0].plot(raw_detection_df.frame.unique(), n_feature_per_frame, '.')
    ax[0, 0].set(xlabel = 'Frame', ylabel = 'N of droplets', title = 'N of droplets per frame')
    ax[0, 1].plot(raw_detection_df.r, '.')
    ax[0, 1].set(xlabel = 'Feature index', ylabel = 'Radius [px]', title = 'Radius of features detected')
    ax[1, 0].scatter(raw_detection_df.r, raw_detection_df.eccentricity, s=0.1)
    ax[1, 0].set(xlabel = 'Radius [px]', ylabel='Eccentricity', title='R-eccentricity correlation')
    ax[1, 1].scatter(raw_detection_df.r, raw_detection_df.prob, s=0.1)
    ax[1, 1].set(xlabel = 'Radius [px]', ylabel='Probability', title='R-Probability correlation')
    plt.tight_layout()
    plt.savefig(save_path + f'raw_features_{model_name}_{startFrame}_{endFrame}.png', dpi = 500)
    plt.close()
else:
    raw_detection_df = pd.read_parquet(save_path + f'raw_detection_25b25r-1_modified_2D_versatile_fluo_{startFrame}_{endFrame}.parquet')
    sample_frames = raw_detection_df.frame.unique()

In [10]:
err_frames = np.where(raw_detection_df.groupby('frame').count().x.values != nDrops)[0]
print(f'Number of errors: {len(err_frames)} / {len(sample_frames)} --> {len(err_frames)/len(sample_frames)*100:.2f}%')
condition = np.ediff1d(err_frames)
condition[condition == 1] = True
condition[condition != 1] = False
max_n_of_consecutive_errs = max(np.diff(np.where(np.concatenate(([condition[0]], condition[:-1] != condition[1:], [True])))[0])[::2])
print(f'Max number of consecutive errors: {max_n_of_consecutive_errs}')

Number of errors: 618 / 540000 --> 0.11%
Max number of consecutive errors: 15


In [11]:
if link_verb:
    print('Linking trajectories...')
    cutoff = 100
    mem = max_n_of_consecutive_errs + 1
    t = tp.link_df(raw_detection_df, cutoff, memory = mem, link_strategy = 'hybrid', neighbor_strategy = 'KDTree', adaptive_stop = 1)
    #print(t)
    t = t.sort_values(['frame', 'particle'])
    trajectories = tp.filter_stubs(t, 25)
    # CREATE COLOR COLUMN AND SAVE DF
    n = max(trajectories.particle)
    print(f'N of droplets: {n + 1}')
    random.seed(5)
    colors = ['#'+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(n)]
    for i in range(max(trajectories.particle)+1-n):
        colors.append('#00FFFF')
    c = []
    for p in t.particle:
        c.append(colors[p])
    trajectories['color'] = c
    trajectories = trajectories.reset_index(drop=True)
    trajectories.to_parquet(save_path + f'raw_tracking_25b25r-1_modified_2D_versatile_fluo_{startFrame}_{endFrame}.parquet', index = False)
else:
    print('Importing linked trajectories...')
    trajectories = pd.read_parquet(save_path + f'raw_tracking_25b25r-1_modified_2D_versatile_fluo_{startFrame}_{endFrame}.parquet')

Linking trajectories...


In [ ]:
if interp_verb:
    print('Interpolating trajectories...')
    interp_trajectories = trajectories.groupby('particle').apply(interpolate_trajectory)
    interp_trajectories = interp_trajectories.reset_index(drop=True)
    interp_trajectories['particle'] = interp_trajectories['particle'].astype(int)
    interp_trajectories = interp_trajectories.sort_values(['frame', 'particle'])
    interp_trajectories.to_parquet(save_path + f'interpolated_tracking_25b25r-1_modified_2D_versatile_fluo_{startFrame}_{endFrame}.parquet', index=False)
else:
    print('Importing interpolated trajectories...')
    interp_trajectories = pd.read_parquet(save_path + f'interpolated_tracking_25b25r-1_modified_2D_versatile_fluo_{startFrame}_{endFrame}.parquet')

Importing interpolated trajectories...


In [9]:
traj_test = interp_trajectories.loc[interp_trajectories.frame.isin(err_frames)]
fig, ax = fig, ax = plt.subplots(1, 1, figsize=(10, 10))
def update_graph(frame):
    df = traj_test.loc[(traj_test.frame == frame), ["x", "y", "r", 'color']]
    for i in range(len(df)):
        graph[i].center = (df.x.values[i], df.y.values[i])
        graph[i].radius = df.r.values[i]
    graph2.set_data(get_frame(video, frame, xmin, ymin, xmax, ymax, w, h, False))
    title.set_text(f'{system_name} Tracking -- t = {round(frame/fps, 1)} s')
    return graph

title = ax.set_title(f'{system_name} Tracking -- t = {0} s')
ax.set(xlabel = 'X [px]', ylabel = 'Y [px]')
df = traj_test.loc[(traj_test.frame == min(traj_test.frame.unique())), ["x", "y", "r", 'color']]
graph = []
for i in range(len(df)):
    graph.append(ax.add_artist(plt.Circle((df.x.values[i], df.y.values[i]), df.r.values[i], color = df.color.values[i],\
                                           fill = True, alpha = 0.5, linewidth=1)))
graph2 = ax.imshow(get_frame(video, 0, xmin, ymin, xmax, ymax, w, h, False))
ani = matplotlib.animation.FuncAnimation(fig, update_graph, traj_test.frame.unique(), interval = 5, blit=False)
writer = matplotlib.animation.FFMpegWriter(fps = 30, metadata = dict(artist='Matteo Scandola'), extra_args=['-vcodec', 'libx264'])
ani.save(f'./{save_path}/tracking_video_errors.mp4', writer=writer, dpi = 200)
plt.close()